In [1]:
import numpy as np
from numpy.linalg import inv
import math
import pandas 
import random 
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement, combinations, product

In [2]:
dataframe1 = pandas.read_csv("data_X.csv")
dataframe2 = pandas.read_csv("data_T.csv")

X_all = np.array(dataframe1.values, dtype = np.float)
Y_all = np.array(dataframe2.values, dtype = np.float)

c = list(zip(X_all, Y_all))
random.shuffle(c)
X_all, Y_all = zip(*c)

X_all = np.array(X_all)
Y_all = np.array(Y_all)

X_all.reshape(500,8)
Y_all.reshape(500,2)

X_test = np.array(X_all[400:500,1:8], dtype = np.float)
Y_test = np.array(Y_all[400:500,1:2], dtype = np.float)
X_test.reshape(100,7)
Y_test.reshape(100,1)

X_train = np.array(X_all[:400,1:8], dtype = np.float)
Y_train = np.array(Y_all[:400,1:2], dtype = np.float)
X_train.reshape(400,7)
Y_train.reshape(400,1)

array([[0.92],
       [0.82],
       [0.87],
       [0.42],
       [0.65],
       [0.85],
       [0.76],
       [0.57],
       [0.49],
       [0.81],
       [0.7 ],
       [0.62],
       [0.64],
       [0.61],
       [0.73],
       [0.93],
       [0.81],
       [0.92],
       [0.91],
       [0.71],
       [0.7 ],
       [0.86],
       [0.56],
       [0.74],
       [0.94],
       [0.73],
       [0.91],
       [0.72],
       [0.81],
       [0.49],
       [0.78],
       [0.72],
       [0.86],
       [0.76],
       [0.36],
       [0.6 ],
       [0.87],
       [0.64],
       [0.82],
       [0.69],
       [0.7 ],
       [0.71],
       [0.62],
       [0.71],
       [0.75],
       [0.34],
       [0.39],
       [0.47],
       [0.86],
       [0.74],
       [0.93],
       [0.82],
       [0.85],
       [0.95],
       [0.57],
       [0.83],
       [0.62],
       [0.79],
       [0.77],
       [0.71],
       [0.62],
       [0.65],
       [0.89],
       [0.66],
       [0.57],
       [0.92],
       [0.

In [3]:
def RMSE(x, w, t):
    y = np.dot(x, w)
    return math.sqrt(np.power(y-t, 2).sum()/len(x))

def train(x, y, lambda0 = 0): 
    w = np.linalg.pinv(x.T.dot(x) + lambda0 * np.identity(len(x.T.dot(x)))).dot(x.T).dot(y)
    return w

def regression_makingPhi(x, M):
    init1 = np.full((len(x), 1), 1)      
    for M1 in range(M):
        for c in list(product(range(len(x[0])), repeat=M1+1)):
            p = np.expand_dims(np.prod(x[:, c], axis=1), axis=1)
            init1 = np.concatenate((init1, p), axis=1)
    return init1
    
def gaussion_makingPhi(x, meanX, varX):
    phi = []     
    for featureIndex in range(len(meanX)):
        thisColume = x[:,featureIndex]
        oneExpW = []
        for x0Index in range(len(x)):                
            oneExpW.append(math.exp(-1 *  ((thisColume[x0Index] - meanX[featureIndex]) ** 2)/ (2 * (varX[featureIndex] ** 2))))
        phi.append(oneExpW)
    phi = np.array(phi).T     
    return phi

In [4]:
lambdas = [0.01,0.1,1,10,100]
numberOfFold = 5
X_num = len(X_all)
oneFoldNum = int(X_num / numberOfFold)
for foldIndex in range(numberOfFold):
    X_test = np.array(X_all[foldIndex*oneFoldNum:(foldIndex+1)*oneFoldNum,1:8], dtype = np.float)
    Y_test = np.array(Y_all[foldIndex*oneFoldNum:(foldIndex+1)*oneFoldNum,1:2], dtype = np.float)
    X_test.reshape(oneFoldNum,7)
    Y_test.reshape(oneFoldNum,1)
    if(foldIndex == 0):
        X_train = np.array(X_all[:X_num-oneFoldNum,1:8], dtype = np.float)
        Y_train = np.array(Y_all[:X_num-oneFoldNum,1:2], dtype = np.float)
    elif(foldIndex == numberOfFold-1 ):
        X_train = np.array(X_all[oneFoldNum:X_num,1:8], dtype = np.float)
        Y_train = np.array(Y_all[oneFoldNum:X_num,1:2], dtype = np.float)
    else:
        X_train = np.array(X_all[0:foldIndex*oneFoldNum,1:8], dtype = np.float)
        X_train = np.append(X_train,np.array(X_all[(foldIndex+1)*oneFoldNum:X_num,1:8], dtype = np.float) , axis = 0)
        Y_train = np.array(Y_all[0:foldIndex*oneFoldNum,1:2], dtype = np.float)
        Y_train = np.append(Y_train,np.array(Y_all[(foldIndex+1)*oneFoldNum:X_num,1:2], dtype = np.float) , axis = 0)        
    X_train.reshape(X_num - oneFoldNum,7)
    Y_train.reshape(X_num - oneFoldNum,1)
    
    trainRmse = []
    testRmse = []
    M_max = 3
    

    for m in range(1, M_max+1): 
        phi = regression_makingPhi(X_train,m)    
        w = train(phi, Y_train)
        RMSE0 = RMSE(phi, w, Y_train)
        trainRmse.append(RMSE0)


        phii = regression_makingPhi(X_test,m)    
        RMSE0 = RMSE(phii, w, Y_test)
        testRmse.append(RMSE0)
        
    print("fold:",foldIndex)
    for i in range(1, M_max+1):
        print("M=%d, training rmse: %f, testing rmse: %f" % (i, trainRmse[i-1], testRmse[i-1]))
    print()

fold: 0
M=1, training rmse: 0.058340, testing rmse: 0.059739
M=2, training rmse: 0.055318, testing rmse: 0.055947
M=3, training rmse: 0.054817, testing rmse: 0.056588

fold: 1
M=1, training rmse: 0.060117, testing rmse: 0.057257
M=2, training rmse: 0.057382, testing rmse: 0.059071
M=3, training rmse: 0.056722, testing rmse: 0.059652

fold: 2
M=1, training rmse: 0.060204, testing rmse: 0.057140
M=2, training rmse: 0.057294, testing rmse: 0.057628
M=3, training rmse: 0.056077, testing rmse: 0.062778

fold: 3
M=1, training rmse: 0.058719, testing rmse: 0.063873
M=2, training rmse: 0.056021, testing rmse: 0.064791
M=3, training rmse: 0.055103, testing rmse: 0.068421

fold: 4
M=1, training rmse: 0.059435, testing rmse: 0.063194
M=2, training rmse: 0.056997, testing rmse: 0.060899
M=3, training rmse: 0.055944, testing rmse: 0.060265



In [5]:
for lambdaa in lambdas:
    for foldIndex in range(numberOfFold):
        X_test = np.array(X_all[foldIndex*oneFoldNum:(foldIndex+1)*oneFoldNum,1:8], dtype = np.float)
        Y_test = np.array(Y_all[foldIndex*oneFoldNum:(foldIndex+1)*oneFoldNum,1:2], dtype = np.float)
        X_test.reshape(oneFoldNum,7)
        Y_test.reshape(oneFoldNum,1)
        if(foldIndex == 0):
            X_train = np.array(X_all[:X_num-oneFoldNum,1:8], dtype = np.float)
            Y_train = np.array(Y_all[:X_num-oneFoldNum,1:2], dtype = np.float)
        elif(foldIndex == numberOfFold-1 ):
            X_train = np.array(X_all[oneFoldNum:X_num,1:8], dtype = np.float)
            Y_train = np.array(Y_all[oneFoldNum:X_num,1:2], dtype = np.float)
        else:
            X_train = np.array(X_all[0:foldIndex*oneFoldNum,1:8], dtype = np.float)
            X_train = np.append(X_train,np.array(X_all[(foldIndex+1)*oneFoldNum:X_num,1:8], dtype = np.float) , axis = 0)
            Y_train = np.array(Y_all[0:foldIndex*oneFoldNum,1:2], dtype = np.float)
            Y_train = np.append(Y_train,np.array(Y_all[(foldIndex+1)*oneFoldNum:X_num,1:2], dtype = np.float) , axis = 0)        
        X_train.reshape(X_num - oneFoldNum,7)
        Y_train.reshape(X_num - oneFoldNum,1)

        trainRmse = []
        testRmse = []
        M_max = 3
       

        for m in range(1, M_max+1):  
            phi = regression_makingPhi(X_train,m)    
            w = train(phi, Y_train,lambdaa)
            RMSE0 = RMSE(phi, w, Y_train)
            trainRmse.append(RMSE0)


            phii = regression_makingPhi(X_test,m)    
            RMSE0 = RMSE(phii, w, Y_test)
            testRmse.append(RMSE0)
        print("fold:",foldIndex," , lamda:",lambdaa)
        for i in range(1, M_max+1):
            print("M=%d, training rmse: %f, testing rmse: %f" % (i, trainRmse[i-1], testRmse[i-1]))
        print()

fold: 0  , lamda: 0.01
M=1, training rmse: 0.058352, testing rmse: 0.059818
M=2, training rmse: 0.055331, testing rmse: 0.055767
M=3, training rmse: 0.054821, testing rmse: 0.056602

fold: 1  , lamda: 0.01
M=1, training rmse: 0.060129, testing rmse: 0.057220
M=2, training rmse: 0.058163, testing rmse: 0.060817
M=3, training rmse: 0.056724, testing rmse: 0.059634

fold: 2  , lamda: 0.01
M=1, training rmse: 0.060216, testing rmse: 0.057170
M=2, training rmse: 0.057301, testing rmse: 0.057626
M=3, training rmse: 0.056100, testing rmse: 0.062501

fold: 3  , lamda: 0.01
M=1, training rmse: 0.058731, testing rmse: 0.063837
M=2, training rmse: 0.056027, testing rmse: 0.064757
M=3, training rmse: 0.055109, testing rmse: 0.068407

fold: 4  , lamda: 0.01
M=1, training rmse: 0.059447, testing rmse: 0.063232
M=2, training rmse: 0.057133, testing rmse: 0.061151
M=3, training rmse: 0.055941, testing rmse: 0.060256

fold: 0  , lamda: 0.1
M=1, training rmse: 0.059029, testing rmse: 0.060935
M=2, train

In [6]:
for c in list(combinations(range(0, 7), 6)): 
    phi = regression_makingPhi(X_train[:, list(c)],1)
    w = train(phi, Y_train)
    RMSE0 = RMSE(phi, w, Y_train)

    print("usingIndexes: ", list(c), "\nRMSE: ", RMSE0)
    print()

usingIndexes:  [0, 1, 2, 3, 4, 5] 
RMSE:  0.06019104005159794

usingIndexes:  [0, 1, 2, 3, 4, 6] 
RMSE:  0.06792038812277285

usingIndexes:  [0, 1, 2, 3, 5, 6] 
RMSE:  0.060465289249398056

usingIndexes:  [0, 1, 2, 4, 5, 6] 
RMSE:  0.05944160080722656

usingIndexes:  [0, 1, 3, 4, 5, 6] 
RMSE:  0.05970566574205993

usingIndexes:  [0, 2, 3, 4, 5, 6] 
RMSE:  0.05979675158375489

usingIndexes:  [1, 2, 3, 4, 5, 6] 
RMSE:  0.060275007588020756



In [7]:
XtrainMean = []
XtrainVar = []
for feature in range(X_train.shape[1]):
    temm = 0
    for studentIndex in range(len(X_train)):
        temm += X_train[studentIndex][feature]
    mean0 = temm / len(X_train)
    XtrainMean.append(mean0)

    temm = 0
    for studentIndex in range(len(X_train)):
        temm += (X_train[studentIndex][feature] - mean0) ** 2
    var0 = math.sqrt(temm / len(X_train))
    XtrainVar.append(var0)
XtrainMean = np.array(XtrainMean)
XtrainVar = np.array(XtrainVar)

phi = gaussion_makingPhi(X_train, XtrainMean, XtrainVar)
w = train(phi, Y_train)
RMSE0 = RMSE(phi, w, Y_train)

phii = gaussion_makingPhi(X_test, XtrainMean, XtrainVar)
RMSE1 = RMSE(phii, w, Y_test)

print("rmse_train_ML: ",RMSE0)
print("rmse_test_ML: ",RMSE1)

rmse_train_ML:  0.12122834787018699
rmse_test_ML:  0.1227206011885074


In [8]:
for lambdaa in lambdas:
    
    phi = gaussion_makingPhi(X_train, XtrainMean, XtrainVar)
    w = train(phi, Y_train,lambdaa)
    RMSE0 = RMSE(phi, w, Y_train)

    phii = gaussion_makingPhi(X_test, XtrainMean, XtrainVar)
    RMSE1 = RMSE(phii, w, Y_test)

    print("lambda = ", lambdaa," , rmse_train_MAP:",RMSE0)
    print("lambda = ", lambdaa," , rmse_test_MAP:",RMSE1)
    print()

lambda =  0.01  , rmse_train_MAP: 0.12122844622640036
lambda =  0.01  , rmse_test_MAP: 0.12270558623052376

lambda =  0.1  , rmse_train_MAP: 0.12123803367743766
lambda =  0.1  , rmse_test_MAP: 0.12257766161815002

lambda =  1  , rmse_train_MAP: 0.12206228819084303
lambda =  1  , rmse_test_MAP: 0.12191664711830928

lambda =  10  , rmse_train_MAP: 0.14777080451490776
lambda =  10  , rmse_test_MAP: 0.13454249692478076

lambda =  100  , rmse_train_MAP: 0.2149658365067034
lambda =  100  , rmse_test_MAP: 0.17872823830242246

